Author: Kevin ALBERT  

Created: March 2021

### Import open-source packages

In [1]:
# environment packages
import platform
import psutil
import os

# other packages
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', 100) # default 50, the maximum width in characters of a column
pd.set_option('display.max_columns', 40)   # default 20, the maximum amount of columns in view 
pd.set_option('display.max_rows', 60)      # default 60, the maximum amount of rows in view
import logging
import json
import requests
import joblib

### Import azure machine learning SDK packages

In [2]:
from azureml.core import Workspace, Dataset, Datastore, Run
from azureml.core.experiment import Experiment
from azureml.data.datapath import DataPath
from azureml.core.compute import ComputeTarget, AmlCompute, AksCompute
from azureml.core.model import Model, InferenceConfig
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from azureml.widgets import RunDetails
from azureml.core.webservice import Webservice, AciWebservice, AksWebservice
from azureml.exceptions import WebserviceException
from azureml.core.environment import Environment
from azureml.train.estimator import Estimator
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling, GridParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core.runconfig import RunConfiguration
from azureml.pipeline.core import PipelineData, Pipeline
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep
from azureml.pipeline.core.run import PipelineRun
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.interpret import ExplanationClient
import azureml.core
print("azureml.core version:", azureml.core.__version__)

azureml.core version: 1.24.0


### Workspace

In [3]:
# load the workspace
ws = Workspace.from_config()

### Experiment

In [60]:
# choose an experiment name
experiment = Experiment(ws, 'automl-binary-classification-SDSHackathon')

In [61]:
# dataset = pd.read_parquet("../../../data/platinum/dataset.parquet")
# dataset = pd.read_parquet("../../../data/silver/ultraPlusSpotify.parquet")
dataset = pd.read_parquet("../../../data/silver/ultraPlusSpotifyplusPDF.parquet")

In [62]:
# dataset.columns

In [45]:
# dataset.columns

Index(['#', 'Artiste', 'Titre', 'fileDate', 'freq', 'True_Target',
       'CleanedTitle', 'id', 'popularity', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms'],
      dtype='object')

In [63]:
dataset.columns

Index(['index', '#_x', 'Artiste', 'Titre', 'fileDate_x', 'freq', 'True_Target',
       'CleanedTitle', 'id', 'popularity', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', '#_y', 'VW', 'W',
       'Artiest', 'Titel', 'Label', 'Distributeur', 'HP', 'fileDate_y'],
      dtype='object')

In [22]:
# dataset.sample(3)

,True_Target,length,popularity,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
50,True,228983,1,0.15,0.56,0.60,0.00,0.07,-6.16,0.06,104.76
41,True,278719,5,0.05,0.45,0.58,0.00,0.08,-6.76,0.03,139.63
13,True,219800,5,0.01,0.47,0.94,0.00,0.24,-3.68,0.05,75.05


In [46]:
# dataset.iloc[[50,41,13]]

,#,Artiste,Titre,fileDate,freq,True_Target,CleanedTitle,id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
50,36,Aya Nakamura,Jolie nana,20-11-21,2,False,Jolie nana,65RWVU6N81CeH65nu52K1U,72,0.81,0.64,2,-5.91,1,0.27,0.27,0.00,0.13,0.49,92.99,147076
41,42,Julien Doré,La fièvre,20-11-14,1,False,La fièvre,79jeTvkW3gvUdldDkQpYOf,62,0.80,0.65,2,-7.93,0,0.07,0.60,0.01,0.10,0.41,89.98,241547
13,14,Ariana Grande,Positions,20-11-14,17,True,positions,35mvY5S1H3J2QZyna3TFe0,93,0.74,0.80,0,-4.77,1,0.09,0.47,0.00,0.09,0.68,144.01,172325


In [64]:
dataset.iloc[[50,41,13]]

,index,#_x,Artiste,Titre,fileDate_x,freq,True_Target,CleanedTitle,id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,#_y,VW,W,Artiest,Titel,Label,Distributeur,HP,fileDate_y
50,50,36,Aya Nakamura,Jolie nana,20-11-21,2,False,Jolie nana,65RWVU6N81CeH65nu52K1U,72,0.81,0.64,2,-5.91,1,0.27,0.27,0.00,0.13,0.49,92.99,147076,49,37,18,Aya Nakamura,Jolie nana,REC. 118,WARNER,8,28_11_2020
41,41,42,Julien Doré,La fièvre,20-11-14,1,False,La fièvre,79jeTvkW3gvUdldDkQpYOf,62,0.80,0.65,2,-7.93,0,0.07,0.60,0.01,0.10,0.41,89.98,241547,None,None,None,None,None,None,None,None,None
13,13,14,Ariana Grande,Positions,20-11-14,17,True,positions,35mvY5S1H3J2QZyna3TFe0,93,0.74,0.80,0,-4.77,1,0.09,0.47,0.00,0.09,0.68,144.01,172325,42,36,14,Ariana Grande,Positions,REPUBLIC,UNIVERSAL,14,30_01_2021


In [65]:
# dataset.drop(["#","Artiste_x","Titre","fileDate","freq","album","Artiste_y","release_date","time_signature"], axis=1, inplace=True)
# dataset.drop(["#","fileDate","Artiste","Titre","freq","CleanedTitle","id"], axis=1, inplace=True)
dataset.drop(["index","#_x","#_y","fileDate_x","fileDate_y","Artiste","Titre","freq","CleanedTitle","id","Artiest","Titel","Label","Distributeur","HP","W"], axis=1, inplace=True)

In [66]:
# we propose to use these features:
dataset.iloc[[50,41,13]]

,True_Target,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,VW
50,False,72,0.81,0.64,2,-5.91,1,0.27,0.27,0.00,0.13,0.49,92.99,147076,37
41,False,62,0.80,0.65,2,-7.93,0,0.07,0.60,0.01,0.10,0.41,89.98,241547,None
13,True,93,0.74,0.80,0,-4.77,1,0.09,0.47,0.00,0.09,0.68,144.01,172325,36


In [25]:
# dataset["True_Target"].sum()

43

In [67]:
dataset["True_Target"].sum()

89

In [68]:
dataset.shape

(118, 15)

In [7]:
# we are not going to split for now... use the whole dataset

## Train

In [70]:
automl_settings = {
    "enable_early_stopping":False,
    "experiment_timeout_hours":4,
    "iterations":250, # number of runs
    "iteration_timeout_minutes":5,
    "max_concurrent_iterations":1,
    "max_cores_per_iteration":-1,
    #"experiment_exit_score":0.9920,
    "model_explainability":True,
    "n_cross_validations":5,
    "primary_metric":'precision_score_weighted', # we could use recall_score_weighted, AUC_weighted
    "featurization":'auto',
    "verbosity":logging.INFO, # {INFO, DEBUG, CRITICAL, ERROR, WARNING} -- debug_log=<*.log>
}

automl_config = AutoMLConfig(task='classification',
                             debug_log='automl_errors.log',
                             compute_target='local', # {training_cluster or 'local'}
                             #blacklist_models=['KNN','LinearSVM'],
                             enable_onnx_compatible_models=False,
                             training_data=dataset,
                             label_column_name="True_Target", # the name of the target variable
                             **automl_settings
                            )
# ouputs "model.pkl" and "automl_errors.log"

In [71]:
automl_run = experiment.submit(automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_2f7095c6-38ae-48c6-a1b5-6047a04fa9bc

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing f

        65   MaxAbsScaler LogisticRegression                0:00:33       0.7592    0.8040
        66   StandardScalerWrapper LogisticRegression       0:00:33       0.7988    0.8040
        67   StandardScalerWrapper LogisticRegression       0:00:33       0.7455    0.8040
        68   MinMaxScaler LogisticRegression                0:00:33       0.7794    0.8040
        69   StandardScalerWrapper XGBoostClassifier        0:00:40       0.6832    0.8040
        70   StandardScalerWrapper LogisticRegression       0:00:34       0.7455    0.8040
        71   StandardScalerWrapper LogisticRegression       0:00:33       0.7455    0.8040
        72   MaxAbsScaler LogisticRegression                0:00:42       0.7896    0.8040
        73   StandardScalerWrapper LogisticRegression       0:00:35       0.7917    0.8040
        74   StandardScalerWrapper ExtremeRandomTrees       0:00:35       0.6560    0.8040
        75   StandardScalerWrapper LogisticRegression       0:00:36       0.7602    0.8040

       155   MinMaxScaler LogisticRegression                0:00:34       0.7592    0.8117
       156   StandardScalerWrapper LogisticRegression       0:00:35       0.7850    0.8117
       157   RobustScaler LogisticRegression                0:00:40       0.7331    0.8117
       158   StandardScalerWrapper LogisticRegression       0:00:34       0.7963    0.8117
       159   MinMaxScaler SVM                               0:00:34       0.6036    0.8117
       160   StandardScalerWrapper LogisticRegression       0:00:34       0.7470    0.8117
       161   StandardScalerWrapper GradientBoosting         0:00:34       0.7217    0.8117
       162   MaxAbsScaler LogisticRegression                0:00:41       0.7493    0.8117
       163   StandardScalerWrapper XGBoostClassifier        0:00:40       0.6872    0.8117
       164   MaxAbsScaler LogisticRegression                0:00:33       0.7846    0.8117
       165   StandardScalerWrapper LogisticRegression       0:00:34       0.7412    0.8117

       245   MaxAbsScaler LogisticRegression                0:00:36       0.7592    0.8117
       246   MaxAbsScaler LogisticRegression                0:00:41       0.7331    0.8117
       247   MaxAbsScaler LogisticRegression                0:00:34       0.7846    0.8117
       248   VotingEnsemble                                 0:01:40       0.8117    0.8117
       249   StackEnsemble                                  0:01:24       0.6875    0.8117
****************************************************************************************************
Current status: BestRunExplainModel. Best run model explanations started
Current status: ModelExplanationDataSetSetup. Model explanations data setup completed
Current status: PickSurrogateModel. Choosing LightGBM as the surrogate model for explanations
Current status: EngineeredFeatureExplanations. Computation of engineered features started
Current status: EngineeredFeatureExplanations. Computation of engineered features completed
Current s

### explore the best pipeline

In [72]:
RunDetails(automl_run).show()
automl_run.wait_for_completion() # get more parameter info

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_2f7095c6-38ae-48c6-a1b5-6047a04fa9bc',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-03-14T01:51:34.091918Z',
 'endTimeUtc': '2021-03-14T04:42:02.583362Z',
 'properties': {'num_iterations': '250',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'precision_score_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-binary-classification-SDSHackathon","subscription_id":"43c1f93a-903d-4b23-a4bf-92bd7a150627","resource_group":"myResourceGroup11","workspace_name":"machine_learning_workspace11","region":"northeurope","compute_target":"local","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":250,"primary_metric":"precision_score_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross

### select best pipeline 

In [73]:
best_run, fitted_model = automl_run.get_output()

### inspect model properties

In [74]:
# pipeline steps
for step in fitted_model.named_steps:
    print(step)

datatransformer
StandardScalerWrapper
LogisticRegression


In [75]:
# model properties
fitted_model.named_steps

{'datatransformer': DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                 feature_sweeping_config=None, feature_sweeping_timeout=None,
                 featurization_config=None, force_text_dnn=None,
                 is_cross_validation=None, is_onnx_compatible=None, logger=None,
                 observer=None, task=None, working_dir=None),
 'StandardScalerWrapper': <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper at 0x7efbfe2b40f0>,
 'LogisticRegression': LogisticRegression(C=6866.488450042998, class_weight=None, dual=False,
                    fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                    max_iter=100, multi_class='ovr', n_jobs=-1, penalty='l2',
                    random_state=None, solver='saga', tol=0.0001, verbose=0,
                    warm_start=False)}

In [76]:
# show all metrics
best_run.get_metrics()

{'recall_score_weighted': 0.8144927536231885,
 'matthews_correlation': 0.4597739096154541,
 'f1_score_weighted': 0.7972343201404986,
 'AUC_weighted': 0.7463659147869673,
 'AUC_macro': 0.7463659147869673,
 'f1_score_macro': 0.709097888834731,
 'average_precision_score_micro': 0.8365347676405861,
 'accuracy': 0.8144927536231885,
 'precision_score_macro': 0.7906390977443609,
 'log_loss': 0.5857079592448764,
 'norm_macro_recall': 0.36651039363113663,
 'average_precision_score_macro': 0.7501837333335268,
 'f1_score_micro': 0.8144927536231885,
 'weighted_accuracy': 0.8893599170307847,
 'AUC_micro': 0.8396778512917453,
 'recall_score_macro': 0.6832551968155683,
 'precision_score_micro': 0.8144927536231885,
 'precision_score_weighted': 0.8117297773419054,
 'average_precision_score_weighted': 0.8190065259422254,
 'recall_score_micro': 0.8144927536231885,
 'balanced_accuracy': 0.6832551968155683,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_2f7095c6-38ae-48c6-a1b5-6047a04fa9bc_

### quick feature importance checks

In [77]:
client = ExplanationClient.from_run(best_run)
engineered_explanations = client.download_model_explanation(raw=False)
feature_importance = engineered_explanations.get_feature_importance_dict() # get model feature importance values
columns = ["modelFeatureImportance_name", "modelFeatureImportance_value"]
pd.DataFrame(list(feature_importance.items()), columns=columns)

,modelFeatureImportance_name,modelFeatureImportance_value
0,acousticness_MeanImputer,0.55
1,duration_ms_MeanImputer,0.55
2,speechiness_MeanImputer,0.52
3,valence_MeanImputer,0.30
4,liveness_MeanImputer,0.28
5,popularity_MeanImputer,0.25
6,tempo_MeanImputer,0.14
7,energy_MeanImputer,0.11
8,key_MeanImputer,0.10
9,instrumentalness_MeanImputer,0.06


## Register

####  prepare the scoring script, environment file and model

In [79]:
# get the score and environment files
model_name = best_run.properties['model_name'] # score.py script will look for the name of the registered model

# make a local copy of the best scoring script, environment file and the model file
script_file_name = 'inference/score.py'
conda_env_file_name = 'inference/env.yml'
model_pickle_file_name = 'inference/model.pkl'
# model_onnx_file_name = 'inference/model.onnx'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file_name)
best_run.download_file('outputs/conda_env_v_1_0_0.yml', conda_env_file_name)
best_run.download_file('outputs/model.pkl', model_pickle_file_name)
# best_run.download_file('outputs/model.onnx', model_onnx_file_name)

In [80]:
! cat inference/env.yml







name: project_environment
dependencies:


- python=3.6.2

- pip:
  - azureml-train-automl-runtime==1.24.0
  - inference-schema
  - azureml-interpret==1.24.0
  - azureml-defaults==1.24.0
- numpy>=1.16.0,<1.19.0
- pandas==0.25.1
- scikit-learn==0.22.1
- py-xgboost<=0.90
- fbprophet==0.5
- holidays==0.9.11
- psutil>=5.2.2,<6.0.0
channels:
- anaconda
- conda-forge


In [81]:
! cat inference/score.py

# ---------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# ---------------------------------------------------------
import json
import logging
import os
import pickle
import numpy as np
import pandas as pd
import joblib

import azureml.automl.core
from azureml.automl.core.shared import logging_utilities, log_server
from azureml.telemetry import INSTRUMENTATION_KEY

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType


input_sample = pd.DataFrame({"VW": pd.Series(["example_value"], dtype="object"), "acousticness": pd.Series([0.0], dtype="float64"), "danceability": pd.Series([0.0], dtype="float64"), "duration_ms": pd.Series([0], dtype="int64"), "energy": pd.Series([0.0], dtype="float64"), "instrumentalness": pd.Series([0.0], dtype="f

### Register the model 

#### we use the local /path/model folder with Model.register()

In [82]:
model = Model.register(workspace=ws,
                       model_name=model_name, # registered model name used in scoring script init()
                       model_framework=Model.Framework.SCIKITLEARN, # {TensorFlow, ScikitLearn, Onnx, Custom}
                       model_framework_version='0.22.2',
                       model_path='inference/model.pkl', # local file {'model.pkl', 'model.onnx'}
                       tags={'Training context': 'autoML Training'},
                       properties={'AUC': best_run.get_metrics()['AUC_weighted'],
                                   'Accuracy': best_run.get_metrics()['accuracy']},
                       description="Classification model to predict diabetes")

Registering model AutoML2f7095c63110


### Deploy model as webservice (ACI)

#### this deplayment can take 15-20min... (using smallest compute 1CPU, 1GB mem)

In [ ]:
%%time
# Configure the scoring environment
service_name = "automl-projname-service" # only lowercase letters, numbers, or dashes

# Remove any existing service under the same name
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    print('"' + service_name + '" does not exist, creating the webservice...')

myenv = Environment.from_conda_specification(name="myenv", file_path=conda_env_file_name)
inference_config = InferenceConfig(entry_script=script_file_name, environment=myenv)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                                       memory_gb=1)

# build container from environment, start webservice ACI and deploy inference scrips 
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)
service.wait_for_deployment(show_output=True)

"automl-projname-service" does not exist, creating the webservice...
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-03-14 09:03:26+01:00 Creating Container Registry if not exists..
2021-03-14 09:03:42+01:00 Registering the environment..
2021-03-14 09:03:43+01:00 Building image.

In [ ]:
# get webservice logs
print(service.get_logs())

Send a HTTP triggered webrequest with testdata to the model for a prediction value.  
In this example we test a person is inTop50 (1) or not-Top50 (0).  
The data must be a list of 11 features to predict a binary classification.  

In [ ]:
# get webservice URI
endpoint = service.scoring_uri

# raw test data
rawdata = [[5, 2, 180, 74, 24, 21, 24, 1.5, 22, , ],
           [6, 0, 148, 58, 11, 179, 39, 0.16, 45, , ]]

print("URI: " + endpoint)
print("Body: " + json.dumps({"data": rawdata})) # convert array to a serialized JSON formatted string object

In [ ]:
response = requests.post(endpoint, json={"data": rawdata})
response.json()